In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

d:\App\anaconda\envs\toolkit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
#movies data
movies = tfds.load("movielens/100k-movies", split="train")
#ratings data
ratings = tfds.load("movielens/100k-ratings", split="train")

In [27]:
def sort_ratings(ds):
    timestamps = []
    features_list = []

    for example in ds:
        timestamps.append(example['timestamp'])
        features_list.append(example)

    # convert list to tensor
    timestamps = tf.convert_to_tensor(timestamps)
    features_list = tf.stack(features_list)
    # sort index
    sorted_indices = tf.argsort(timestamps)
    # sort all data
    sorted_features = tf.gather(ds, sorted_indices)
    return tf.data.Dataset.from_tensor_slices(sorted_features)

In [31]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [32]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [51]:
movie_titles = movies.batch(2_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', ..., b'Zeus and Roxanne (1997)',
       b'unknown', b'\xc3\x81 k\xc3\xb6ldum klaka (Cold Fever) (1994)'],
      dtype=object)

In [ ]:
unique_movie_titles = np.unique(list(movie_titles))
unique_user_ids = np.unique(list(user_ids))

unique_movie_titles[:10]